In [18]:
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

In [20]:

import os
from typing import List, Optional
from dataclasses import dataclass
import pdfplumber
import docx
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from datetime import datetime


In [21]:
@dataclass
class InterviewQuestionRequest:
    """면접 질문 생성 요청 데이터"""
    position: str  # 지원 직무
    company: str   # 회사명
    prompt: str    # 사용자 입력 프롬프트
    # resume_content: str  # 이력서 내용

In [22]:
class DocumentProcessor:
    """문서 처리 클래스"""
    
    @staticmethod
    def extract_pdf_text(file_path: str) -> str:
        try:
            text = ""
            with pdfplumber.open(file_path) as pdf:
                for page in pdf.pages:
                    text += page.extract_text()
            return text
        except Exception as e:
            raise Exception(f"PDF 파일 처리 오류: {str(e)}")
    
    @staticmethod
    def extract_docx_text(file_path: str) -> str:
        """DOCX 파일에서 텍스트 추출"""
        try:
            doc = docx.Document(file_path)
            text = ""
            for paragraph in doc.paragraphs:
                text += paragraph.text + "\n"
            return text
        except Exception as e:
            raise Exception(f"DOCX 파일 처리 오류: {str(e)}")
    
    @staticmethod
    def extract_text_from_file(file_path: str) -> str:
        """파일 확장자에 따라 텍스트 추출"""
        if file_path.lower().endswith('.pdf'):
            return DocumentProcessor.extract_pdf_text(file_path)
        elif file_path.lower().endswith('.docx'):
            return DocumentProcessor.extract_docx_text(file_path)
        elif file_path.lower().endswith('.txt'):
            with open(file_path, 'r', encoding='utf-8') as file:
                return file.read()
        else:
            raise ValueError("지원되지 않는 파일 형식입니다. PDF, DOCX, TXT만 지원됩니다.")

In [ ]:
class InterviewQuestionGenerator:
    """면접 질문 생성기"""
    
    def __init__(self, api_key: str, model_name: str = "gpt-4o-mini"):
        """
        초기화
        Args:
            api_key: OpenAI API 키
            model_name: 사용할 모델 (gpt-3.5-turbo-0125 또는 gpt-4o-mini)
        """
        self.llm = ChatOpenAI(
            api_key=api_key,
            model=model_name,
            temperature=0.7,
            max_tokens=2000
        )
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=2000,
            chunk_overlap=200
        )
        
    def _create_prompt_template(self) -> ChatPromptTemplate:
        """프롬프트 템플릿 생성"""
        template = """
당신은 면접관이며, 지원자의 이력서와 지원 정보를 바탕으로 적절한 면접 질문을 생성해야 합니다.

[지원 정보]
- 지원 직무: {position}
- 회사명: {company}
- 추가 요구사항: {prompt}

[이력서 내용]

위 정보를 바탕으로 다음 카테고리별로 면접 질문을 생성해주세요:

1. **기본 질문** (2-3개): 자기소개, 지원동기 등
2. **경험 기반 질문** (3-4개): 이력서의 경험을 바탕으로 한 구체적 질문
3. **기술적 질문** (2-3개): 직무 관련 기술적 역량 확인
4. **상황 대응 질문** (2-3개): 문제 해결, 팀워크 등 상황별 대응 능력
5. **회사/직무 적합성 질문** (2개): 회사와 직무에 대한 이해도 확인

각 질문은 다음 형식으로 작성해주세요:
- 질문 내용을 명확하게 작성
- 필요시 질문의 의도나 평가 포인트를 간단히 설명

총 10-15개의 질문을 생성해주세요.
"""
        
        return ChatPromptTemplate.from_template(template)
    
    def generate_questions(self, request: InterviewQuestionRequest) -> str:
        """면접 질문 생성"""
        try:
            # 프롬프트 템플릿 생성
            prompt_template = self._create_prompt_template()
            
            # 체인 구성
            chain = prompt_template | self.llm | StrOutputParser()
            
            # 질문 생성
            result = chain.invoke({
                "position": request.position,
                "company": request.company,
                "prompt": request.prompt,
                # "resume_content": request.resume_content
            })
            
            return result
            
        except Exception as e:
            raise Exception(f"질문 생성 오류: {str(e)}")
    
    def generate_follow_up_questions(self, original_question: str, answer: str) -> str:
        """답변 기반 추가 질문 생성"""
        follow_up_template = """
면접관으로서, 다음 질문에 대한 지원자의 답변을 바탕으로 2-3개의 후속 질문을 생성해주세요.

원래 질문: {original_question}
지원자 답변: {answer}

후속 질문은 다음을 고려해서 생성해주세요:
- 답변의 구체성 확인
- 더 깊이 있는 경험 탐색
- 관련된 다른 상황에서의 대응 능력 확인

각 후속 질문마다 왜 이 질문을 하는지 간단한 의도도 함께 설명해주세요.
"""
        
        template = ChatPromptTemplate.from_template(follow_up_template)
        chain = template | self.llm | StrOutputParser()
        
        return chain.invoke({
            "original_question": original_question,
            "answer": answer
        })


In [24]:
def main():
    """메인 실행 함수"""
    # 환경 변수에서 API 키 가져오기
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        print("OPENAI_API_KEY 환경 변수를 설정해주세요.")
        return
    
    # 질문 생성기 초기화
    generator = InterviewQuestionGenerator(
        api_key=api_key,
        model_name="gpt-3.5-turbo-0125"  # 또는 "gpt-3.5-turbo-0125","gpt-4o-mini" 등 필요에 따라 변경 가능
    )
    
    # 사용자 입력 받기
    print("=== 면접 예상 질문 생성기 ===")
    position = input("지원 직무를 입력하세요: ")
    company = input("회사명을 입력하세요: ")
    prompt = input("추가 요구사항이나 특별히 고려할 점을 입력하세요: ")
    
    # 이력서 파일 처리
    # file_path = input("이력서 파일 경로를 입력하세요 (PDF/DOCX/TXT): ")
    
    try:
        # 이력서 내용 추출
        # resume_content = DocumentProcessor.extract_text_from_file(file_path)
        
        # 요청 객체 생성
        request = InterviewQuestionRequest(
            position=position,
            company=company,
            prompt=prompt,
            # resume_content=resume_content
        )
        
        # 면접 질문 생성
        print("\n질문을 생성하고 있습니다...")
        questions = generator.generate_questions(request)
        
        # 결과 출력
        print("\n" + "="*60)
        print("📋 생성된 면접 예상 질문")
        print("="*60)
        print(questions)
        
        # 결과를 파일로 저장
        output_file = f"면접질문_{company}_{position}.txt"
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(f"회사: {company}\n")
            f.write(f"직무: {position}\n")
            f.write(f"생성일: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write("\n" + "="*60 + "\n")
            f.write(questions)
        
        print(f"\n✅ 결과가 '{output_file}' 파일에 저장되었습니다.")
        
    except Exception as e:
        print(f"❌ 오류 발생: {str(e)}")

In [27]:
if __name__ == "__main__":
    # 필요한 패키지 설치 안내
    required_packages = [
        "langchain-openai",
        "langchain-core", 
        "PyPDF2",
        "python-docx"
    ]
    
    print("필요한 패키지 설치:")
    print("pip install " + " ".join(required_packages))
    print("-" * 50)
    
    main()

필요한 패키지 설치:
pip install langchain-openai langchain-core PyPDF2 python-docx
--------------------------------------------------
=== 면접 예상 질문 생성기 ===

질문을 생성하고 있습니다...

📋 생성된 면접 예상 질문
1. **기본 질문**
- 자기소개 부탁드립니다.
- 의도: 지원자의 경력과 능력을 간단하게 소개받아 볼 수 있습니다.

- 네이버 백엔드 개발자로 지원한 이유가 무엇인가요?
- 의도: 지원자가 회사에 대한 관심과 동기를 파악할 수 있습니다.

2. **경험 기반 질문**
- 이력서에 기재된 백엔드 개발 경험 중에서 가장 도전적이었던 프로젝트는 무엇이었나요? 어떻게 해결했나요?
- 의도: 지원자의 문제해결 능력과 도전정신을 확인할 수 있습니다.

- 장교 복무 경험을 바탕으로 팀워크를 발휘한 경험을 언급해주실 수 있을까요?
- 의도: 지원자의 팀워크 및 리더십 능력을 평가할 수 있습니다.

3. **기술적 질문**
- 데이터베이스 관련 기술에 대해 어떤 경험이 있으신가요? 어떤 데이터베이스를 주로 활용하시나요?
- 의도: 지원자의 기술 역량과 전문 지식을 확인할 수 있습니다.

- 네이버의 서비스를 개발하면서 어떤 백엔드 기술 스택을 활용하고 싶으신가요?
- 의도: 회사와 직무에 대한 이해도와 기술적 역량을 평가할 수 있습니다.

4. **상황 대응 질문**
- 프로젝트에서 예상치 못한 문제가 발생했을 때 어떻게 대처하셨나요?
- 의도: 문제 해결 능력과 스트레스 관리 능력을 확인할 수 있습니다.

- 팀원들과 갈등 상황이 발생했을 때 어떻게 대응했는지 알려주세요.
- 의도: 팀워크와 커뮤니케이션 능력을 평가할 수 있습니다.

5. **회사/직무 적합성 질문**
- 네이버에서 백엔드 개발자로 일하면서 어떤 역할을 하고 싶으신가요?
- 의도: 지원자의 역량과 회사 직무에 대한 이해도를 확인할 수 있습니다.

- 네이버의 백엔드 개발자로서 어떤 